In [1]:
import asyncio
import csv
import enum
import logging
import queue
import threading
import numpy as np

from typing import Callable, Dict, List, Optional, TextIO

from ready_trader_go.match_events import MatchEvents
from ready_trader_go.order_book import IOrderListener, Order, OrderBook
from ready_trader_go.types import Instrument, Lifespan, Side

MARKET_EVENT_QUEUE_SIZE = 1024 
INPUT_SCALING = 100


class MarketEventOperation(enum.IntEnum):
    AMEND = 0
    CANCEL = 1
    INSERT = 2
    Amend = AMEND
    Cancel = CANCEL
    Insert = INSERT


class MarketEvent(object):
    """A market event."""
    __slots__ = ("time", "instrument", "operation", "order_id", "side", "volume", "price", "lifespan")

    def __init__(self, time: float, instrument: Instrument, operation: MarketEventOperation, order_id: int,
                 side: Optional[Side], volume: int, price: int, lifespan: Optional[Lifespan]):
        """Initialise a new instance of the MarketEvent class."""
        self.time: float = time
        self.instrument: Instrument = instrument
        self.operation: MarketEventOperation = operation
        self.order_id: int = order_id
        self.side: Optional[Side] = side
        self.volume: int = volume
        self.price: int = price
        self.lifespan: Optional[Lifespan] = lifespan


class MarketEventsReader(IOrderListener):
    """A processor of market events read from a file."""

    def __init__(self, filename: str, future_book: OrderBook, etf_book: OrderBook, match=False):
        """Initialise a new instance of the MarketEvents class.
        """
        self.etf_book: OrderBook = etf_book
        self.etf_orders: Dict[int, Order] = dict()
        self.filename: str = filename
        self.future_book: OrderBook = future_book
        self.future_orders: Dict[int, Order] = dict()
        self.logger: logging.Logger = logging.getLogger("MARKET_EVENTS")
        self.queue: queue.Queue = queue.Queue(MARKET_EVENT_QUEUE_SIZE)
        self.reader_task: Optional[threading.Thread] = None
        self.match = match

        # Prime the event pump with a no-op event
        self.next_event: Optional[MarketEvent] = MarketEvent(0.0, Instrument.FUTURE, MarketEventOperation.CANCEL, 0,
                                                             Side.BUY, 0, 0, Lifespan.FILL_AND_KILL)

        # Allow other objects to get a callback when the reader task is complete
        self.task_complete: List[Callable] = list()

    # IOrderListener callbacks

    def on_order_amended(self, now: float, order: Order, volume_removed: int) -> None:
        """Called when the order is amended."""
        if order.remaining_volume == 0:
            if order.instrument == Instrument.FUTURE:
                del self.future_orders[order.client_order_id]
            elif order.instrument == Instrument.ETF:
                del self.etf_orders[order.client_order_id]

    def on_order_cancelled(self, now: float, order: Order, volume_removed: int) -> None:
        """Called when the order is cancelled."""
        if order.instrument == Instrument.FUTURE and order.client_order_id in self.future_orders:
            del self.future_orders[order.client_order_id]
        elif order.instrument == Instrument.ETF and order.client_order_id in self.etf_orders:
            del self.etf_orders[order.client_order_id]

    def on_order_placed(self, now: float, order: Order) -> None:
        """Called when a good-for-day order is placed in the order book."""
        if order.instrument == Instrument.FUTURE:
            self.future_orders[order.client_order_id] = order
        elif order.instrument == Instrument.ETF:
            self.etf_orders[order.client_order_id] = order

    def on_order_filled(self, now: float, order: Order, price: int, volume: int, fee: int) -> None:
        """Called when the order is partially or completely filled."""
        if order.remaining_volume == 0:
            if order.instrument == Instrument.FUTURE and order.client_order_id in self.future_orders:
                del self.future_orders[order.client_order_id]
            elif order.instrument == Instrument.ETF and order.client_order_id in self.etf_orders:
                del self.etf_orders[order.client_order_id]

    def on_reader_done(self, num_events: int) -> None:
        """Called when the market data reader thread is done."""
        self.logger.info("reader thread complete after processing %d market events", num_events)

    def process_market_events(self, elapsed_time: float) -> None:
        """Process market events from the queue."""
        evt: MarketEvent = self.next_event

        while evt and evt.time < elapsed_time:
            if evt.instrument == Instrument.FUTURE:
                orders = self.future_orders
                book = self.future_book
            else:
                orders = self.etf_orders
                book = self.etf_book

            if evt.operation == MarketEventOperation.INSERT:
                order = Order(evt.order_id, evt.instrument, evt.lifespan, evt.side, evt.price, evt.volume, self)
                book.insert(evt.time, order)
            elif evt.order_id in orders:
                order = orders[evt.order_id]
                if evt.operation == MarketEventOperation.CANCEL:
                    book.cancel(evt.time, order)
                elif evt.volume < 0:
                    # evt.operation must be MarketEventOperation.AMEND
                    book.amend(evt.time, order, order.volume + evt.volume)

            evt = self.queue.get()

        self.next_event = evt
        if evt is None:
            for c in self.task_complete:
                c(self)

    def reader(self, market_data: TextIO) -> None:
        """Read the market data file and place order events in the queue."""
        fifo = self.queue

        with market_data:
            csv_reader = csv.reader(market_data)
            next(csv_reader)  # Skip header row
            for row in csv_reader:
                # time, instrument, operation, order_id, side, volume, price, lifespan
                fifo.put(MarketEvent(float(row[0]), Instrument(int(row[1])), MarketEventOperation[row[2]],
                                     int(row[3]), Side[row[4]] if row[4] else None,
                                     int(float(row[5])) if row[5] else 0, int(float(row[6]) * INPUT_SCALING) if row[6] else 0,
                                     Lifespan[row[7]] if row[7] else None))
            fifo.put(None)

    def start(self):
        """Start the market events reader thread"""
        try:
            market_data = open(self.filename)
        except OSError as e:
            self.logger.error("failed to open market data file: filename='%s'" % self.filename, exc_info=e)
            raise
        else:
            self.reader_task = threading.Thread(target=self.reader, args=(market_data,), daemon=True, name="reader")
            self.reader_task.start()


In [2]:
file = 'data/tournament2_files/market_data_tournament2.csv'
file = 'data/mock_t2.csv'
# with open(file, 'r') as f:
#     file_len = len(f.readlines())

future_book = OrderBook(Instrument.FUTURE, 0.0, 0.0)
etf_book = OrderBook(Instrument.ETF, -0.0001, 0.0002)

mer = MarketEventsReader(filename=file, future_book=future_book, etf_book=etf_book, match=False)
mer.start()

In [3]:
mer.process_market_events(0.05)
print(str(mer.etf_book))

BidVol	Price	AskVol
	62500c	  1000
  1000	61400c


In [4]:
mer.process_market_events(0.35)
print(str(mer.etf_book))

BidVol	Price	AskVol
	62500c	  1000
	62400c	    72
  1000	61400c
    14	60800c
    37	60600c


In [5]:
so = Order(client_order_id = 1, instrument = Instrument.ETF, lifespan = Lifespan.G, side = Side.BUY, price = 62700, volume = 900)
etf_book.insert(0.3502, order = so)
print(str(mer.etf_book))

Passive has been filled, id: 5023033 - now, passive, best_price, volume, fee (0.3502, <ready_trader_go.order_book.Order object at 0x0000022B2DDC8A50>, 62400, 72, -449) 
Aggressor has been filled, id: 1 - now, passive, best_price, volume, fee (0.3502, <ready_trader_go.order_book.Order object at 0x0000022B2DDC8BA0>, 62400, 72, 899) 
Passive has been filled, id: 7080584 - now, passive, best_price, volume, fee (0.3502, <ready_trader_go.order_book.Order object at 0x0000022B2DDC8660>, 62500, 828, -5175) 
Aggressor has been filled, id: 1 - now, passive, best_price, volume, fee (0.3502, <ready_trader_go.order_book.Order object at 0x0000022B2DDC8BA0>, 62500, 828, 10350) 
BidVol	Price	AskVol
	62500c	   172
  1000	61400c
    14	60800c
    37	60600c


In [6]:
so = Order(client_order_id = 344, instrument = Instrument.ETF, lifespan = Lifespan.G, side = Side.BUY, price = 61400, volume = 900)
etf_book.insert(0.3502, order = so)

In [7]:
print(str(mer.etf_book))

BidVol	Price	AskVol
	62500c	   172
  1900	61400c
    14	60800c
    37	60600c


In [8]:
mer.process_market_events(0.87)

In [3]:
print(str(mer.etf_book))

BidVol	Price	AskVol




In [10]:
so = Order(client_order_id = 345, instrument = Instrument.ETF, lifespan = Lifespan.G, side = Side.SELL, price = 60800, volume = 300)
etf_book.insert(0.4502, order = so)

Passive has been filled, id: 7080585 - now, passive, best_price, volume, fee (0.4502, <ready_trader_go.order_book.Order object at 0x0000022B2DDC86D0>, 61400, 300, -1842) 
Aggressor has been filled, id: 345 - now, passive, best_price, volume, fee (0.4502, <ready_trader_go.order_book.Order object at 0x0000022B2DD56350>, 61400, 300, 3684) 


In [11]:
so = Order(client_order_id = 346, instrument = Instrument.ETF, lifespan = Lifespan.G, side = Side.SELL, price = 60800, volume = 1000)
etf_book.insert(0.4502, order = so)
print(str(mer.etf_book))

Passive has been filled, id: 7080585 - now, passive, best_price, volume, fee (0.4502, <ready_trader_go.order_book.Order object at 0x0000022B2DDC86D0>, 61400, 700, -4298) 
Passive has been filled, id: 344 - now, passive, best_price, volume, fee (0.4502, <ready_trader_go.order_book.Order object at 0x0000022B2DDC8CF0>, 61400, 300, -1842) 
Aggressor has been filled, id: 346 - now, passive, best_price, volume, fee (0.4502, <ready_trader_go.order_book.Order object at 0x0000022B2DD56510>, 61400, 1000, 12280) 


In [13]:
print(str(mer.etf_book))

BidVol	Price	AskVol
	62600c	    34
	62500c	   172
	62000c	    21
   600	61400c
    14	60800c
    37	60600c


In [14]:
so = Order(client_order_id = 347, instrument = Instrument.ETF, lifespan = Lifespan.G, side = Side.SELL, price = 61400, volume = 600)
etf_book.insert(0.4502, order = so)
print(str(mer.etf_book))

Passive has been filled, id: 344 - now, passive, best_price, volume, fee (0.4502, <ready_trader_go.order_book.Order object at 0x0000022B2DDC8CF0>, 61400, 600, -3684) 
Aggressor has been filled, id: 347 - now, passive, best_price, volume, fee (0.4502, <ready_trader_go.order_book.Order object at 0x0000022B2DD56890>, 61400, 600, 7368) 
BidVol	Price	AskVol
	62600c	    34
	62500c	   172
	62000c	    21
    14	60800c
    37	60600c


In [15]:
so = Order(client_order_id = 348, instrument = Instrument.ETF, lifespan = Lifespan.G, side = Side.BUY, price = 61300, volume = 524)
etf_book.insert(0.88, order = so)
print(str(mer.etf_book))

BidVol	Price	AskVol
	62600c	    34
	62500c	   172
	62000c	    21
   524	61300c
    14	60800c
    37	60600c


In [18]:
mer.process_market_events(0.9)
print(str(mer.etf_book))

Passive has been filled, id: 348 - now, passive, best_price, volume, fee (0.899898, <ready_trader_go.order_book.Order object at 0x0000022B2DD564A0>, 61300, 524, -3212) 
Aggressor has been filled, id: 7080586 - now, passive, best_price, volume, fee (0.899898, <ready_trader_go.order_book.Order object at 0x0000022B2DE8A040>, 61300, 524, 6424) 
BidVol	Price	AskVol
	62600c	    34
	62000c	    21
	61300c	   476
    14	60800c
    37	60600c
  1000	60200c


In [19]:
so = Order(client_order_id = 349, instrument = Instrument.ETF, lifespan = Lifespan.F, side = Side.BUY, price = 61000, volume = 524)
etf_book.insert(0.88, order = so)
print(str(mer.etf_book))

BidVol	Price	AskVol
	62600c	    34
	62000c	    21
	61300c	   476
    14	60800c
    37	60600c
  1000	60200c


In [4]:
## PLACE AUTOTRADERS
# chacnges u must make:

## overwrute the three send order methods
## create a self.order_queue = queue.Queue(1024) class var
## change all instances of next(self.order_ids) to -> next(self.order_ids) * 10 + REV_CFG[self.team_name]['uid']
## ensure team name matches config files
## Change OrderBook and Order defined classes if u have them


# Copyright 2021 Optiver Asia Pacific Pty. Ltd.
#
# This file is part of Ready Trader Go.
#
#     Ready Trader Go is free software: you can redistribute it and/or
#     modify it under the terms of the GNU Affero General Public License
#     as published by the Free Software Foundation, either version 3 of
#     the License, or (at your option) any later version.
#
#     Ready Trader Go is distributed in the hope that it will be useful,
#     but WITHOUT ANY WARRANTY; without even the implied warranty of
#     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#     GNU Affero General Public License for more details.
#
#     You should have received a copy of the GNU Affero General Public
#     License along with Ready Trader Go.  If not, see
#     <https://www.gnu.org/licenses/>.
from ast import Or
import asyncio
import itertools
import numpy as np
from typing import List, NamedTuple

from ready_trader_go import BaseAutoTrader, Instrument, Lifespan, MAXIMUM_ASK, MINIMUM_BID, Side


LOT_SIZE = 10
POSITION_LIMIT = 100
TICK_SIZE_IN_CENTS = 100

Order_C = NamedTuple('Order_C', [('price', int), ('vol', int), ('id', int)])
PotentialVolume = NamedTuple('PotentialVolume', [('max', int), ('min', int)])
Hedge = NamedTuple('Hedge', [('ETF', NamedTuple), ('FUTURE', NamedTuple)])

class OrderBook_C:
    def __init__(self, sequence_number: int, ask_prices: List[int], ask_volumes: List[int], bid_prices: List[int], bid_volumes: List[int]):
        self.sequence_number = sequence_number
        self.bids = [Order_C(price=bid_prices[i], vol=bid_volumes[i], id=-1) for i in range(len(bid_prices))]
        self.asks = [Order_C(price=ask_prices[i], vol=ask_volumes[i], id=-1) for i in range(len(ask_prices))]
        self.best_bid = self.bids[0]
        self.best_ask = self.asks[0]

class Historical:
    def __init__(self):
        self.historical_prices = {
            Instrument.FUTURE: [], # Future
            Instrument.ETF: []  # ETF
        }

    def update(self, instrument, price):
        self.historical_prices[instrument].append(price)

    @property
    def min_time(self):
        return min(
            len(self.historical_prices[Instrument.FUTURE]), 
            len(self.historical_prices[Instrument.ETF]))

    def min_len_to_execute(self, result, length=10, safe_result=0):
        if self.min_time >= length:
            return result
        return safe_result

    @property
    def history_future(self):
        return self.historical_prices[Instrument.FUTURE][:self.min_time]

    @property
    def history_etf(self):
        return self.historical_prices[Instrument.ETF][:self.min_time]

    @property
    def std_etf(self):
        return self.min_len_to_execute(np.std(self.history_etf), safe_result=1)

    @property
    def std_future(self):
        return self.min_len_to_execute(np.std(self.history_future), safe_result=1)

    @property
    def corr(self):
        return self.min_len_to_execute(np.corrcoef(self.history_etf, self.history_future)[0][1])
    
    @property
    def cov(self):
        return self.min_len_to_execute(self.corr * self.std_etf * self.std_future)

    @property
    def beta(self):
        """beta of stock against future"""
        return self.min_len_to_execute(self.cov / (self.std_future**2))
    

class ArbV1(BaseAutoTrader):
    """Example Auto-trader.

    When it starts this auto-trader places ten-lot bid and ask orders at the
    current best-bid and best-ask prices respectively. Thereafter, if it has
    a long position (it has bought more lots than it has sold) it reduces its
    bid and ask prices. Conversely, if it has a short position (it has sold
    more lots than it has bought) then it increases its bid and ask prices.
    """

    def __init__(self, loop: asyncio.AbstractEventLoop, team_name: str, secret: str):
        """Initialise a new instance of the AutoTrader class."""
        super().__init__(loop, team_name, secret)
        self.order_ids = itertools.count(1)
        
        self.bids = {} # current
        self.bid_orders = {} # filled orders uses hedge id as key
        self.future_bids = set()
        
        self.asks = {}
        self.ask_orders = {} # filled orders
        self.future_asks = set()
        
        self.arbitrage_hedges = set()
        
        self.ask_id = self.ask_price = self.bid_id = self.bid_price = self.position = self.futures_position = 0
        
        self.historical = Historical()

        self.order_queue = queue.Queue(1024)

    @property
    def potential_position(self):
        return PotentialVolume(max = self.position + sum([order.vol for order in self.bids.values()]), min = self.position - sum([order.vol for order in self.asks.values()]))

    def send_insert_order(self, client_order_id ,side ,price ,volume ,lifespan):
        """Oerwrites normal command"""
        self.order_queue.put(Order(client_order_id = client_order_id, instrument = Instrument.ETF, lifespan = lifespan, side = side, price = price, volume = volume))

    def send_hedge_order(self, client_order_id, side, price, volume):
        """Overwrites normal command"""
        self.order_queue.put(Order(client_order_id = client_order_id, instrument = Instrument.FUTURE, lifespan = Lifespan.F, side = side, price = price, volume = volume))

    def send_cancel_order(self, client_order_id):
        """Overwrites normal command"""
        
        self.order_queue.put(Order(client_order_id = client_order_id, instrument = Instrument.FUTURE, lifespan = Lifespan.F, side = side, price = price, volume = volume))

    def get_total_position(self):
        return self.position + self.futures_position

    def on_error_message(self, client_order_id: int, error_message: bytes) -> None:
        """Called when the exchange detects an error.

        If the error pertains to a particular order, then the client_order_id
        will identify that order, otherwise the client_order_id will be zero.
        """
        self.logger.warning("error with order %d: %s", client_order_id, error_message.decode())
        if client_order_id != 0:
            self.on_order_status_message(client_order_id, 0, 0, 0)

    def on_hedge_filled_message(self, client_order_id: int, price: int, volume: int) -> None:
        """Called when one of your hedge orders is filled, partially or fully.

        The price is the average price at which the order was (partially) filled,
        which may be better than the order's limit price. The volume is
        the number of lots filled at that price.

        If the order was unsuccessful, both the price and volume will be zero.
        """
        self.logger.info("received hedge filled for order %d with average price %d and volume %d", client_order_id,
                         price, volume)
        if client_order_id in self.future_asks: # Bought ETF, Sold Future
            self.futures_position -= volume
            if price <= self.bid_orders[client_order_id].ETF.price:
                self.logger.info(f"UNFAVOURABLE ARBITRAGE: BOUGHT ETF AT {self.bid_orders[client_order_id].ETF.price}, SOLD FUTURE AT {price}")
                return
        if client_order_id in self.future_bids: # Sold ETF, Bought Future
            self.futures_position += volume
            if price >= self.ask_orders[client_order_id].ETF.price:
                self.logger.info(f"UNFAVOURABLE ARBITRAGE: SOLD ETF AT {self.ask_orders[client_order_id].ETF.price}, BOUGHT FUTURE AT {price}")
            return

    def on_order_book_update_message(self, instrument: int, sequence_number: int, ask_prices: List[int],
                                     ask_volumes: List[int], bid_prices: List[int], bid_volumes: List[int]) -> None:
        """Called periodically to report the status of an order book.

        The sequence number can be used to detect missed or out-of-order
        messages. The five best available ask (i.e. sell) and bid (i.e. buy)
        prices are reported along with the volume available at each of those
        price levels.
        """
        self.logger.info("received order book for instrument %d with sequence number %d", instrument,
                         sequence_number)
        # self.logger.info(f"TIME: {self.event_loop.time()}")
        # adds to historical prices (mid price)
        self.historical.update(instrument, np.mean([bid_prices[0], ask_prices[0]]))
        if instrument == Instrument.FUTURE:
            # Load Order_C book into memory
            self.futures = OrderBook_C(sequence_number, ask_prices,ask_volumes, bid_prices, bid_volumes)   
        elif instrument == Instrument.ETF:
            # Load Order_C book into memory
            self.etfs = OrderBook_C(sequence_number, ask_prices, ask_volumes, bid_prices, bid_volumes)
            ## ARBITRAGE:
            if self.etfs.best_ask.price < self.futures.best_bid.price: # etf buy, future sell opp.
                # All favourable etf ask prices to trade with
                arb_asks = [ask for ask in self.etfs.asks if ask.price < self.futures.best_bid.price]
                # Keeping potential counts so that we can track our position before official fills
                potential_fut_vol = self.futures.best_bid.vol
                for etf_ask in arb_asks:
                    trade_vol = min(POSITION_LIMIT-self.potential_position.max, etf_ask.vol, potential_fut_vol)
                    if trade_vol > 0:
                        potential_fut_vol -= trade_vol
                        next_id = next(self.order_ids) * 10 + REV_CFG[self.team_name]['uid']
                        
                        self.bids[next_id] = Order_C(price=etf_ask.price, vol=trade_vol, id=next_id)
                        self.send_insert_order(client_order_id=next_id, side=Side.BUY, price=etf_ask.price, volume=trade_vol, lifespan=Lifespan.FAK) # buy etf
                        log = {
                            "POSITION": self.position,
                            "POTENTIAL_POSITION": self.potential_position,
                            "FUTURES_POSITION": self.futures_position,
                            "BIDS/ASKS": (self.bids, self.asks),
                            "MAX_VOLUME":  etf_ask.vol,
                            "ACTION": f"BUY {trade_vol} ETF @{etf_ask.price}, ID: {next_id}"
                        }
                        self.logger.info(f"CUSTOM LOG: {log}")

            if self.futures.best_ask.price < self.etfs.best_bid.price: # future buy, etf sell opp.
                arb_bids = [bid for bid in self.etfs.bids if bid.price > self.futures.best_ask.price]
                potential_fut_vol = self.futures.best_bid.vol
                for etf_bid in arb_bids:
                    trade_vol = min(POSITION_LIMIT+self.potential_position.min, etf_bid.vol, potential_fut_vol)
                    if trade_vol > 0:
                        potential_fut_vol -= trade_vol
                        next_id = next(self.order_ids)  * 10 + REV_CFG[self.team_name]['uid']
                        
                        self.asks[next_id] = Order_C(price=etf_bid.price, vol=trade_vol, id=next_id)
                        self.send_insert_order(client_order_id=next_id, side=Side.SELL, price=etf_bid.price, volume=trade_vol, lifespan=Lifespan.FAK) # sell etf
                        log = {
                            "POSITION": self.position,
                            "POTENTIAL_POSITION": self.potential_position,
                            "FUTURES_POSITION": self.futures_position,
                            "BIDS/ASKS": (self.bids, self.asks),
                            "MAX_VOLUME": etf_bid.vol,
                            "ACTION": f"SELL {trade_vol} ETF @{etf_bid.price}, ID: {next_id}"
                        }
                        self.logger.info(f"CUSTOM LOG: {log}")
                        
    def on_order_filled_message(self, client_order_id: int, price: int, volume: int) -> None:
        """Called when when of your orders is filled, partially or fully.

        The price is the price at which the order was (partially) filled,
        which may be better than the order's limit price. The volume is
        the number of lots filled at that price.
        """
        self.logger.info("received order filled for order %d with price %d and volume %d", client_order_id,
                         price, volume)
        if client_order_id in self.bids.keys():
            self.position += volume
            self.bids[client_order_id] = Order_C(id=client_order_id, price=price, vol=(self.bids[client_order_id].vol-volume))
            next_id = next(self.order_ids) * 10 + REV_CFG[self.team_name]['uid']
            self.future_asks.add(next_id)
            self.bid_orders[next_id] = Hedge(ETF = Order_C(price=price, vol=volume, id=client_order_id), FUTURE = Order_C(price=None, vol=volume, id=next_id))
            self.send_hedge_order(client_order_id=next_id, side=Side.SELL, price=MINIMUM_BID, volume=volume) # selling futures
            log = {
                "POSITION": self.position,
                "POTENTIAL_POSITION": self.potential_position,
                "FUTURES_POSITION": self.futures_position,
                "BIDS/ASKS": (self.bids, self.asks),
                "ACTION": f"SELL {volume}x FUTURE @{MINIMUM_BID}, ID: {next_id}"
            }
            self.logger.info(f"CUSTOM LOG: {log}")
        elif client_order_id in self.asks.keys():
            self.position -= volume
            self.asks[client_order_id] = Order_C(id=client_order_id, price=price, vol=(self.asks[client_order_id].vol-volume))
            next_id = next(self.order_ids) * 10 + REV_CFG[self.team_name]['uid']
            self.future_bids.add(next_id)
            self.ask_orders[next_id] = Hedge(ETF = Order_C(price=price, vol=volume, id=client_order_id), FUTURE = Order_C(price=None, vol=volume, id=next_id))
            self.send_hedge_order(client_order_id=next_id, side=Side.BUY, price=MAXIMUM_ASK//TICK_SIZE_IN_CENTS*TICK_SIZE_IN_CENTS, volume=volume) # buying futures
            log = {
                "POSITION": self.position,
                "POTENTIAL_POSITION": self.potential_position,
                "FUTURES_POSITION": self.futures_position,
                "BIDS/ASKS": (self.bids, self.asks),
                "ACTION": f"BUY {volume}x FUTURE @{MAXIMUM_ASK//TICK_SIZE_IN_CENTS*TICK_SIZE_IN_CENTS}, ID: {next_id}"
            }
            self.logger.info(f"CUSTOM LOG: {log}")

    def on_order_status_message(self, client_order_id: int, fill_volume: int, remaining_volume: int,
                                fees: int) -> None:
        """Called when the status of one of your orders changes.

        The fill_volume is the number of lots already traded, remaining_volume
        is the number of lots yet to be traded and fees is the total fees for
        this order. Remember that you pay fees for being a market taker, but
        you receive fees for being a market maker, so fees can be negative.

        If an order is cancelled its remaining volume will be zero.
        """
        self.logger.info("received order status for order %d with fill volume %d remaining %d and fees %d",
                         client_order_id, fill_volume, remaining_volume, fees)

        if remaining_volume == 0:
            if client_order_id in self.bids.keys():
                self.bids.pop(client_order_id)
            elif client_order_id in self.asks.keys():
                self.asks.pop(client_order_id)            
            log = {
                "POSITION": self.position,
                "POTENTIAL_POSITION": self.potential_position,
                "FUTURES_POSITION": self.futures_position,
                "BIDS/ASKS": (self.bids, self.asks)
            }
            self.logger.info(f"CUSTOM LOG: {log}")

    def on_trade_ticks_message(self, instrument: int, sequence_number: int, ask_prices: List[int],
                               ask_volumes: List[int], bid_prices: List[int], bid_volumes: List[int]) -> None:
        """Called periodically when there is trading activity on the market.

        The five best ask (i.e. sell) and bid (i.e. buy) prices at which there
        has been trading activity are reported along with the aggregated volume
        traded at each of those price levels.

        If there are less than five prices on a side, then zeros will appear at
        the end of both the prices and volumes arrays.
        """
        self.logger.info("received trade ticks for instrument %d with sequence number %d", instrument,
                         sequence_number)




In [12]:
def order_book_msg(book, instr):
    ask_prices = [0] * 5
    ask_volumes = [0] * 5
    bid_prices = [0] * 5
    bid_volumes = [0] * 5
    book.top_levels(ask_prices, ask_volumes, bid_prices, bid_volumes)
    
    return {
        "instrument": instr,
        "sequence_number": 0,
        "ask_prices": ask_prices,
        "ask_volumes": ask_volumes,
        "bid_prices": bid_prices,
        "bid_volumes": bid_volumes
    }

REV_CFG = {
    "Arb-v1": {"uid": 1, "latency": 0.005}
}

CONFIG = {value['uid']: {"name": key, "uid": value["uid"], "latency": value["latency"], "autotrader": None} for key, value in REV_CFG.items()}

CONFIG[1]['autotrader'] = ArbV1("", "Arb-v1", "")

etf_orders = {}
future_orders = {}

# for ts in np.arange(0, 3600, 0.05):
for ts in np.arange(0, 5, 0.05):
    mer.process_market_events(ts)
    # read in messages
    while not mer.etf_book.message_queue.empty():
        msg = mer.etf_book.message_queue.get()
        if msg.type == 'status':
            CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].on_order_status_message(**msg.vals)
            
            while not CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].order_queue.empty():
                order_type, order = CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].order_queue.get()
                if order_type == 'etf':
                    mer.etf_book.insert(ts+0.001, order=order)
                elif order_type == 'hedge':
                    mer.future_book.insert(ts+0.001, order=order)
                elif order_type == 'cancel':
                    if order.client_order_id in etf_orders:
                        mer.etf_book.cancel(ts+0.001, order=etf_orders[order])
                    elif order.client_order_id in future_orders:
                        mer.future_book.cancel(ts+0.001, order=future_orders[order])
            
        elif msg.type == 'filled':
            CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].on_order_filled_message(**msg.vals)

            while not CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].order_queue.empty():
                order_type, order = CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].order_queue.get()
                if order_type == 'etf':
                    mer.etf_book.insert(ts+0.001, order=order)
                elif order_type == 'hedge':
                    mer.future_book.insert(ts+0.001, order=order)
                elif order_type == 'cancel':
                    if order.client_order_id in etf_orders:
                        mer.etf_book.cancel(ts+0.001, order=etf_orders[order])
                    elif order.client_order_id in future_orders:
                        mer.future_book.cancel(ts+0.001, order=future_orders[order])

        etf_orders[msg.vals['client_order_id']] = msg.order
        
        # and record PnL
        
    while not mer.future_book.message_queue.empty():
        msg = mer.future_book.message_queue.get()
        if msg.type == 'status':
            CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].on_order_status_message(**msg.vals)

            while not CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].order_queue.empty():
                order_type, order = CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].order_queue.get()
                if order_type == 'etf':
                    mer.etf_book.insert(ts+0.001, order=order)
                elif order_type == 'hedge':
                    mer.future_book.insert(ts+0.001, order=order)
                elif order_type == 'cancel':
                    if order.client_order_id in etf_orders:
                        mer.etf_book.cancel(ts+0.001, order=etf_orders[order])
                    elif order.client_order_id in future_orders:
                        mer.future_book.cancel(ts+0.001, order=future_orders[order])

        elif msg.type == 'filled':
            CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].on_hedge_filled_message(**msg.vals)

            while not CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].order_queue.empty():
                order_type, order = CONFIG[msg.vals['client_order_id'] % 10]['autotrader'].order_queue.get()
                if order_type == 'etf':
                    mer.etf_book.insert(ts+0.001, order=order)
                elif order_type == 'hedge':
                    mer.future_book.insert(ts+0.001, order=order)
                elif order_type == 'cancel':
                    if order.client_order_id in etf_orders:
                        mer.etf_book.cancel(ts+0.001, order=etf_orders[order])
                    elif order.client_order_id in future_orders:
                        mer.future_book.cancel(ts+0.001, order=future_orders[order])

        future_orders[msg.vals['client_order_id']] = msg.order
            
        # record PnL
       
    if ts % 0.25 == 0:
    # receive and exec orders
        time_priority = sorted(list(CONFIG.values()), key=lambda x: x['latency'])
        for bot in time_priority:
            ## Fut
            future_book_msg = order_book_msg(mer.future_book, Instrument.FUTURE)
            CONFIG[bot['uid']]['autotrader'].on_order_book_update_message(**future_book_msg)
            
            ## ETF
            etf_book_msg = order_book_msg(mer.etf_book, Instrument.ETF)
            CONFIG[bot['uid']]['autotrader'].on_order_book_update_message(**etf_book_msg)

            while not CONFIG[bot['uid']]['autotrader'].order_queue.empty():
                order_type, order =  CONFIG[bot['uid']]['autotrader'].order_queue.get()
                if order_type == 'etf':
                    mer.etf_book.insert(ts+0.001, order=order)
                elif order_type == 'hedge':
                    mer.future_book.insert(ts+0.001, order=order)
                elif order_type == 'cancel':
                    if order.client_order_id in etf_orders:
                        mer.etf_book.cancel(ts+0.001, order=order)
                    elif order.client_order_id in future_orders:
                        mer.future_book.cancel(ts+0.001, order=order)

KeyError: 4